In [7]:
import sys
import os
import pandas as pd
import numpy as np
import datetime as dt
import json
import geojson
import requests
import sqlite3 as db
sys.path.append('../')

pd.options.display.max_rows=99

In [8]:
github_userName = 'Tanag3r'
ebird_token = 'j6c7l80ga2ib'
db_name = 'trailheadDirectBirds_sous.db'
##cur = cnx.cursor()

In [9]:
##connect to database
def connectDB():
    try:
        cnx = db.connect(db_name)
    except Exception as cnxError:
        raise UserWarning(f'Unable to connect to database due to: {cnxError}')
    return cnx

NASA Data Products:
- LAI: LAI is a measure for the total area of leaves per unit ground area and directly related to the amount of light that can be intercepted by plants. It is defined as the one-sided green leaf area per unit ground surface area (LAI = leaf area / ground area, m2 / m2) in broadleaf canopies. There are three methods used to measure LAI for conifers; this project uses projected (or one-sided, in accordance the definition for broadleaf canopies) needle area per unit ground area.
    - In general, a higher LAI value indicates more leaf coverage
- fPAR: Fraction of absorbed photosynthetically active radiation (fPAR) is the fraction of incoming solar radiation in the spectrum of 400–700 nm that is absorbed by vegetation canopy. Data is provided as a percentage.
- Land Cover Type 3: Annual Leaf Area Index (LAI) classification
- FAO-Land Cover Classification System 1 (LCCS1) land cover layer

In [10]:
diff = 0.005    ##equivalent to 0.5 miles when applied
hotspotsGeo = pd.DataFrame()
hotspotsGeo = pd.read_sql('SELECT locId,lat,lng FROM Hotspots', con=connectDB())
##TODO #34 remove duplicate hotspots before building table of polygons for each hotspot --DONE
##hotspotsGeo.set_index('index',inplace=True)
hotspotsGeo.sort_values(by=['locId'],ascending=True,inplace=True)
hotspotsGeo.drop_duplicates(subset=['locId'],keep='first',inplace=True)
hotspotsGeo.reset_index()
##TODO #33 write a function to increment the lat & lng values as needed then output the results to a single column in the format lat,lng --DONE

##Define functions to produce a corner of a polygon for each hotspot (NE,SE,SW,NW)
def NW(x,y):
    return x-diff,y+diff
def NE(x,y):
    return x+diff,y+diff
def SE(x,y):
    return x+diff,y-diff
def SW(x,y):
    return x-diff,y-diff

##apply the functions as new columns
##NOTE that appEEARS only accepts coordinates as (longitude,latitude) which is contrary to geoJSON documentation
hotspotsGeo['NW'] = hotspotsGeo.apply(lambda i: NW(i.lng,i.lat), axis = 1)##.astype(str)
hotspotsGeo['NE'] = hotspotsGeo.apply(lambda i: NE(i.lng,i.lat), axis = 1)##.astype(str)
hotspotsGeo['SE'] = hotspotsGeo.apply(lambda i: SE(i.lng,i.lat), axis = 1)##.astype(str)
hotspotsGeo['SW'] = hotspotsGeo.apply(lambda i: SW(i.lng,i.lat), axis = 1)##.astype(str)

In [7]:
##TODO #40 remove the getpass prompt and replace with a credential manager call
earthdata_baseUrl = 'https://lpdaacsvc.cr.usgs.gov/appeears/api/'
appEEARS_username = 'lwylie'
appEEARS_password = 'BdiUPBhUa7ma5ds'
import getpass
NASA_username = getpass.getpass(prompt = 'Enter NASA Earthdata Login Username: ')
NASA_password = getpass.getpass(prompt = 'Enter NASA Earthdata Login Password: ')

In [8]:
##obtain an Earthdata token
##TODO #36 the earthdata API is often under maintenance, write a script to abort this process if a new token cannot be obtain
earthdata_loginURL = 'https://lpdaacsvc.cr.usgs.gov/appeears/api/login'
earthdata_loginRequest = requests.post(earthdata_loginURL,auth=(NASA_username,NASA_password))
earthdata_loginResponse = earthdata_loginRequest.json()
print(earthdata_loginResponse)
##Transcribe token, builder header
earthdata_token = earthdata_loginResponse['token']
earthdata_head = {'Authorization': 'Bearer {}'.format(earthdata_token)}

{'token_type': 'Bearer', 'token': '_XZ-yiaeVWRoKjWtV8d2PpIeqe54GcjjvxGOebN0nnlC7rP4X88SOiCAej_XWHcrAxRO38lCgEZNkOyidkwnZQ', 'expiration': '2022-02-19T18:00:30Z'}


In [15]:
##TODO #43 write a cooked table of locId's with stats to check against list of hotspots. Drive fetch off difference --DONE
#get the earthdata dictionary from the database and return a list of locId's without Prop1 and Type3 data
def list_needMODIS():
    cnx = connectDB()
    try:
        #dataframe of hotspots from the database
        hotspotsGeo = pd.read_sql('SELECT locId,lat,lng FROM Hotspots WHERE locId not in (SELECT DISTINCT locId from earthdata_dictionary)', con=cnx)
        if hotspotsGeo.empty == True:
            raise Exception(f'An empty dataframe has been returned')
        hotspotsGeo.sort_values(by=['locId'],ascending=True,inplace=True)
        hotspotsGeo.drop_duplicates(subset=['locId'],keep='first',inplace=True)
        hotspotsGeo.reset_index()
        #build out squares around each hotspot
        def NW(x,y):
            return x-diff,y+diff
        def NE(x,y):
            return x+diff,y+diff
        def SE(x,y):
            return x+diff,y-diff
        def SW(x,y):
            return x-diff,y-diff
        ##apply the functions as new columns
        ##NOTE that appEEARS only accepts coordinates as (longitude,latitude) which is contrary to geoJSON documentation
        hotspotsGeo['NW'] = hotspotsGeo.apply(lambda i: NW(i.lng,i.lat), axis = 1)##.astype(str)
        hotspotsGeo['NE'] = hotspotsGeo.apply(lambda i: NE(i.lng,i.lat), axis = 1)##.astype(str)
        hotspotsGeo['SE'] = hotspotsGeo.apply(lambda i: SE(i.lng,i.lat), axis = 1)##.astype(str)
        hotspotsGeo['SW'] = hotspotsGeo.apply(lambda i: SW(i.lng,i.lat), axis = 1)##.astype(str)
    
    except Exception as exd:
        raise UserWarning(f'An unexpected error occurred in the function list_needMODIS: {exd}')
    return hotspotsGeo

In [16]:
qa = list_needMODIS()
qa

,locId,lat,lng,NW,NE,SE,SW
0,L10129014,47.536469,-122.039342,"(-122.044342, 47.541469)","(-122.03434200000001, 47.541469)","(-122.03434200000001, 47.531468999999994)","(-122.044342, 47.531468999999994)"
39,L10129054,47.540925,-122.096194,"(-122.10119399999999, 47.545925000000004)","(-122.091194, 47.545925000000004)","(-122.091194, 47.535925)","(-122.10119399999999, 47.535925)"
101,L10265485,47.667353,-122.401002,"(-122.406002, 47.672353300000005)","(-122.39600200000001, 47.672353300000005)","(-122.39600200000001, 47.6623533)","(-122.406002, 47.6623533)"
54,L1034830,47.473939,-122.688961,"(-122.693961, 47.4789392)","(-122.68396100000001, 47.4789392)","(-122.68396100000001, 47.468939199999994)","(-122.693961, 47.468939199999994)"
43,L10445835,47.509108,-122.133556,"(-122.138556, 47.514108)","(-122.128556, 47.514108)","(-122.128556, 47.504107999999995)","(-122.138556, 47.504107999999995)"
82,L1096378,47.660798,-122.373628,"(-122.37862849999999, 47.6657981)","(-122.3686285, 47.6657981)","(-122.3686285, 47.6557981)","(-122.37862849999999, 47.6557981)"
48,L11323884,47.459242,-122.723740,"(-122.7287396, 47.4642424)","(-122.7187396, 47.4642424)","(-122.7187396, 47.4542424)","(-122.7287396, 47.4542424)"
74,L11559026,47.650820,-122.391287,"(-122.3962871, 47.6558198)","(-122.3862871, 47.6558198)","(-122.3862871, 47.6458198)","(-122.3962871, 47.6458198)"
17,L11570424,47.550100,-122.049440,"(-122.05443969999999, 47.5551004)","(-122.0444397, 47.5551004)","(-122.0444397, 47.545100399999995)","(-122.05443969999999, 47.545100399999995)"
19,L1167737,47.539340,-122.061066,"(-122.0660656, 47.5443396)","(-122.05606560000001, 47.5443396)","(-122.05606560000001, 47.534339599999996)","(-122.0660656, 47.534339599999996)"


Architecture:

1. Make all requests in a loop, producing a list of {'locId': 'earthdata_taskID'}
2. For each locId, pull each .csv into a dataframe then load that dataframe into a database table bearing the name that corresponds with the layer and product. Append the locId.
    EXAMPLE: the contents of the .csv file for 'MCD12Q1-006-LC-Prop1-Statistics.csv' go into the table 'MCD12Q1-006-LC-Prop1-Statistics' in the database

In [9]:
##small frame of hotspots for testing
hotspotsGeo_short = hotspotsGeo.head(2)

In [4]:
def geoPack(NW,NE,SE,SW):
    try:
        geoPack_wrap = {"type": "FeatureCollection", "features":
            [{
            "type":"Feature",
                "geometry":
                    {"type": "Polygon",
                    "coordinates":
                        [[NW,NE,SE,SW,NW]]
                    },
                "properties": {}}]
            }
    except geojson.GeoJSON(geoPack_wrap).is_valid == False:
        print(geojson.GeoJSON(geoPack_wrap).errors)
    return geojson.GeoJSON(geoPack_wrap)

In [6]:
def task(taskName: str,endDate: str,startDate: str,recurring: bool,yearRange: list, geoPack_wrap = geojson.GeoJSON):
    try:
        edTask = {
            'task_type': 'area',
            'task_name': taskName,
            'params': {
                'dates': 
                    [{"endDate": endDate, 
                    "recurring": recurring, 
                    "startDate": startDate, 
                    "yearRange": yearRange}],
                'layers': 
                    [{"layer": "LC_Prop1", "product": "MCD12Q1.006"}, 
                    {"layer": "LC_Type3", "product": "MCD12Q1.006"}],
                'output': {
                    'format': {
                        'type': 'netcdf4'}, 
                        'projection': 'geographic'},
            'geo':geoPack_wrap}}
    except Exception as JSONerror:
        raise UserWarning(JSONerror)
    return edTask

In [246]:
def log_earthdataError(message,statusCode: int,request: str,reqDate):
    cnx = connectDB()
    cur = cnx.cursor()
    try:
        sqliteInsert = 'INSERT INTO earthdata_errorlog (message,statusCode,request,reqDate) VALUES (?, ?, ?, ?)'
        logTuple = (message,statusCode,request,reqDate)
        cur.execute(sqliteInsert,logTuple)
        cur.close()
    except db.Error as sqlError:
        raise UserWarning(sqlError)
    finally:
        if cnx:
            cnx.close()

In [5]:
##TODO #45 once all hotspots/locId's have Prop1 and Type3 MCD12Q1 data, convert this block to check for LAI and fPAR
def post_earthdataTask():
    earthdataTaskList = []
    try:
        for locId in hotspotsGeo_short.itertuples():
            coords = geoPack(NW = locId.NW,NE = locId.NE,SE = locId.SE,SW = locId.SW)
            earthdataTask = task(taskName = locId.locId,endDate = "12-31",startDate="01-01",recurring=True,yearRange=[2017,2019],geoPack_wrap=coords)
            taskReq = requests.post(f'{earthdata_baseUrl}task',json=earthdataTask,headers=earthdata_head)
            if taskReq.status_code != 200:
                log_earthdataError(str(taskReq),int(taskReq.status_code),json.dumps(earthdataTask),dt.datetime.today())
            else : earthdataTaskList.append(taskReq.json())
    except Exception as ee:
        raise UserWarning(ee)
    return earthdataTaskList

In [250]:
help = post_earthdataTask()
help

[]

In [105]:
taskid = pd.DataFrame(help)

In [184]:
##submit the task request
##TODO #42 if the request returns anything other than a task_id, stop the script and write the returned issue to the error log
params = {'limit': 15, 'pretty': True}
##earthdata_tasks = requests.get('{}task'.format(earthdata_baseUrl),params=params,headers=earthdata_head).json()
earthdataReq = requests.get('{}task'.format(earthdata_baseUrl),params=params,headers=earthdata_head)
earthdataReq.json()

[{'error': None,
  'params': {'dates': [{'endDate': '12-31',
     'recurring': True,
     'startDate': '01-01',
     'yearRange': [2017, 2019]}],
   'layers': [{'layer': 'LC_Prop1', 'product': 'MCD12Q1.006'},
    {'layer': 'LC_Type3', 'product': 'MCD12Q1.006'}],
   'output': {'format': {'type': 'netcdf4'}, 'projection': 'geographic'}},
  'status': 'done',
  'created': '2022-02-18T19:09:52.459946',
  'task_id': '30a10207-0ba2-4b48-8f48-083c3eb09f1b',
  'updated': '2022-02-18T19:11:33.392441',
  'user_id': 'luke.wylie@comcast.net',
  'attempts': 1,
  'estimate': {'request_size': 778.4074631057572},
  'retry_at': None,
  'completed': '2022-02-18T19:11:33.390705',
  'has_swath': False,
  'task_name': 'L10129002',
  'task_type': 'area',
  'api_version': 'v1',
  'svc_version': '2.70',
  'web_version': None,
  'size_category': '0',
  'has_nsidc_daac': False,
  'expires_on': '2022-03-20T19:11:33.392441'},
 {'error': None,
  'params': {'dates': [{'endDate': '12-31',
     'recurring': True,
    

In [257]:
def get_earthdataTask():
    postedTasks = pd.DataFrame(help)
    taskInfo = []
    fileList = []
    cnx = connectDB()
    ##postedTasks = pd.DataFrame(post_earthdata_Task())
    import time
    try:
        for task_id in postedTasks.itertuples():
            taskStatus = requests.get(f'{earthdata_baseUrl}task/{task_id.task_id}',headers=earthdata_head).json()
            starttime = time.time()
            while taskStatus['status'] !='done':
                print(requests.get(f'{earthdata_baseUrl}task/{task_id.task_id}',headers=earthdata_head).json()['status'])
                time.sleep(20.0 - ((time.time() - starttime) % 20.0))
            taskInfo.append(taskStatus)
        taskBatch = pd.DataFrame(data=taskInfo,columns=['task_id','task_name','status','completed'])

        for task_id in taskBatch.itertuples():
            earthdata_bundle = requests.get(f'{earthdata_baseUrl}bundle/{task_id.task_id}').json()
            ##filter down to .csv files
            for x in earthdata_bundle['files']:
                if x['file_type'] in 'csv':
                    time.sleep(0.3)
                    fileID = x['file_id']
                    fileName = x['file_name']
                    ##To read results to a dataframe, pass in earthdata_baseUrl + 'bundle/ + taskID/ + file_id/ + file_name
                    getIt = pd.DataFrame(pd.read_csv(f'{earthdata_baseUrl}bundle/{task_id.task_id}/{fileID}/{fileName}'))
                    getIt['locId'] = task_id.task_name
                    try:
                        getIt.to_sql(name=f'{fileName}_MODIS_cooking',con=cnx,if_exists='append')
                    ##TODO #53 test the exception
                    except db.DatabaseError:
                        fileList.append({task_id.task_name: f'{earthdata_baseUrl}bundle/{task_id.task_id}/{fileID}/{fileName}'})
                        pass
                else: continue  
    except Exception as ex:
        raise UserWarning(ex)
    cnx.close()
    return fileList

In [258]:
tt = get_earthdataTask()
tt

[]